In [1]:
"""
This example implements the experiments on citation networks from the paper:
Graph Neural Networks with convolutional ARMA filters (https://arxiv.org/abs/1901.01343)
Filippo Maria Bianchi, Daniele Grattarola, Cesare Alippi, Lorenzo Livi
"""

from keras.callbacks import EarlyStopping, TensorBoard, ModelCheckpoint
from keras.layers import Input, Dropout
from keras.models import Model
from keras.optimizers import Adam
from keras.regularizers import l2

from spektral.datasets import citation
from spektral.layers import ARMAConv
from spektral.utils import normalized_laplacian, rescale_laplacian
from spektral.utils.logging import init_logging


Using TensorFlow backend.


In [2]:

# Load data
dataset = 'cora'
adj, node_features, y_train, y_val, y_test, train_mask, val_mask, test_mask = citation.load_data(dataset)


Loading cora dataset


In [32]:
F

1433

In [9]:

# Parameters
ARMA_K = 3                    # Number of parallel ARMA_1 filters
ARMA_D = 2                    # Depth of each ARMA_1 filter
recurrent = True              # Share weights like a recurrent net in each head
N = node_features.shape[0]    # Number of nodes in the graph
F = node_features.shape[1]    # Original feature dimensionality
n_classes = y_train.shape[1]  # Number of classes
dropout_rate = 0.25           # Dropout rate applied to the input of GCN layers
l2_reg = 5e-4                 # Regularization rate for l2
learning_rate = 1e-2          # Learning rate for SGD
epochs = 20               # Number of training epochs
es_patience = 200             # Patience for early stopping
log_dir = init_logging()      # Create log directory and file


<2708x1433 sparse matrix of type '<class 'numpy.float32'>'
	with 49216 stored elements in Compressed Sparse Row format>

In [40]:

# Preprocessing operations
node_features = citation.preprocess_features(node_features)
fltr = normalized_laplacian(adj, symmetric=True)
fltr = rescale_laplacian(fltr, lmax=2)

# Model definition
X_in = Input(shape=(F, ), name="input")
fltr_in = Input((N, ), sparse=True)

dropout_1 = Dropout(dropout_rate)(X_in)
graph_conv_1 = ARMAConv(16,
                        ARMA_K=ARMA_K,
                        ARMA_D=ARMA_D,
                        recurrent=recurrent,
                        dropout_rate=dropout_rate,
                        activation='elu',
                        gcn_activation='elu',
                        kernel_regularizer=l2(l2_reg),
                        use_bias=True)([dropout_1, fltr_in])
dropout_2 = Dropout(dropout_rate)(graph_conv_1)
graph_conv_2 = ARMAConv(n_classes,
                        ARMA_K=1,
                        ARMA_D=1,
                        recurrent=recurrent,
                        dropout_rate=dropout_rate,
                        activation='softmax',
                        gcn_activation=None,
                        kernel_initializer='he_normal',
                        kernel_regularizer=l2(l2_reg),
                        use_bias=True)([dropout_2, fltr_in])

# Build model
model = Model(inputs=[X_in, fltr_in], outputs=graph_conv_2)
optimizer = Adam(lr=learning_rate)
model.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              weighted_metrics=['acc'])
model.summary()



__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              (None, 1433)         0                                            
__________________________________________________________________________________________________
dropout_15 (Dropout)            (None, 1433)         0           input[0][0]                      
__________________________________________________________________________________________________
input_5 (InputLayer)            (None, 2708)         0                                            
__________________________________________________________________________________________________
arma_conv_5 (ARMAConv)          (None, 16)           207216      dropout_15[0][0]                 
                                                                 input_5[0][0]                    
__________

In [41]:
from keras.utils import plot_model

plot_model(model, to_file="semi-super-node-CORA.png", show_shapes=True)

In [11]:
# Callbacks
es_callback = EarlyStopping(monitor='val_weighted_acc', patience=es_patience)
tb_callback = TensorBoard(log_dir=log_dir, batch_size=N, write_graph=True)
mc_callback = ModelCheckpoint(log_dir + 'best_model.h5',
                              monitor='val_weighted_acc',
                              save_best_only=True,
                              save_weights_only=True)

# Train model
validation_data = ([node_features, fltr], y_val, val_mask)
model.fit([node_features, fltr],
          y_train,
          sample_weight=train_mask,
          epochs=epochs,
          batch_size=N,
          validation_data=validation_data,
          shuffle=False,  # Shuffling data means shuffling the whole graph
          callbacks=[es_callback, tb_callback, mc_callback])

# Load best model
model.load_weights(log_dir + 'best_model.h5')


Train on 2708 samples, validate on 2708 samples
Epoch 1/20
2708/2708 [==============================] - 3s 1ms/step - loss: 2.1253 - weighted_acc: 0.0929 - val_loss: 2.0777 - val_weighted_acc: 0.1520
Epoch 2/20
2708/2708 [==============================] - 0s 181us/step - loss: 2.0429 - weighted_acc: 0.3929 - val_loss: 2.0372 - val_weighted_acc: 0.2700
Epoch 3/20
2708/2708 [==============================] - 1s 191us/step - loss: 1.9795 - weighted_acc: 0.5857 - val_loss: 2.0077 - val_weighted_acc: 0.3540
Epoch 4/20
2708/2708 [==============================] - 1s 199us/step - loss: 1.9239 - weighted_acc: 0.5857 - val_loss: 1.9826 - val_weighted_acc: 0.4740
Epoch 5/20
2708/2708 [==============================] - 1s 189us/step - loss: 1.8726 - weighted_acc: 0.7000 - val_loss: 1.9562 - val_weighted_acc: 0.6600
Epoch 6/20
2708/2708 [==============================] - 1s 192us/step - loss: 1.8249 - weighted_acc: 0.8643 - val_loss: 1.9332 - val_weighted_acc: 0.7080
Epoch 7/20
2708/2708 [========

In [33]:

# Evaluate model
print('Evaluating model.')
eval_results = model.evaluate([node_features, fltr],
                              y_test,
                              sample_weight=test_mask,
                              batch_size=N)
print('Done.\n'
      'Test loss: {}\n'
'Test accuracy: {}'.format(*eval_results))

Evaluating model.
2708/2708 [==============================] - 0s 70us/step
Done.
Test loss: 1.641263723373413
Test accuracy: 0.7870006561279297


# MINST 

In [49]:
import numpy as np
import tensorflow as tf
from keras import Input, Model
from keras.callbacks import TensorBoard, EarlyStopping, ModelCheckpoint
from keras.layers import Dense, Flatten
from keras.optimizers import Adam
from keras.regularizers import l2

from spektral.datasets import mnist
from spektral.layers import GraphConv
from spektral.utils import init_logging, normalized_laplacian


def sp_matrix_to_sp_tensor(x):
    x = x.tocoo()
    return tf.SparseTensor(indices=np.array([x.row, x.col]).T,
                           values=x.data, dense_shape=x.shape)


# Parameters
l2_reg = 5e-4             # Regularization rate for l2
learning_rate = 1e-3      # Learning rate for SGD
batch_size = 100           # Batch size
epochs = 1            # Number of training epochs
es_patience = 200         # Patience fot early stopping
log_dir = init_logging()  # Create log directory and file

# Load data
X_train, y_train, X_val, y_val, X_test, y_test, adj = mnist.load_data()

X_train, X_val, X_test = X_train[..., None], X_val[..., None], X_test[..., None]
N = X_train.shape[-2]      # Number of nodes in the graphs
F = X_train.shape[-1]      # Node features dimensionality
n_out = y_train.shape[-1]  # Dimension of the target


In [50]:

fltr = normalized_laplacian(adj)

# Model definition
X_in = Input(shape=(N, F))
# Pass filter as a fixed tensor, otherwise Keras will complain about inputs of
# different rank.
G_in = Input(tensor=sp_matrix_to_sp_tensor(fltr))

graph_conv = GraphConv(32,
                       activation='elu',
                       kernel_regularizer=l2(l2_reg),
                       use_bias=True)([X_in, G_in])

graph_conv = GraphConv(32,
                       activation='elu',
                       kernel_regularizer=l2(l2_reg),
                       use_bias=True)([graph_conv, G_in])
flatten = Flatten()(graph_conv)
fc = Dense(512, activation='relu')(flatten)
output = Dense(n_out, activation='softmax')(fc)

# Build model
model = Model(inputs=[X_in, G_in], outputs=output)
optimizer = Adam(lr=learning_rate)
model.compile(optimizer=optimizer,
              loss='sparse_categorical_crossentropy',
              metrics=['acc'])
model.summary()


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_10 (InputLayer)           (None, 784, 1)       0                                            
__________________________________________________________________________________________________
input_11 (InputLayer)           (784, 784)           0                                            
__________________________________________________________________________________________________
graph_conv_5 (GraphConv)        (None, 784, 32)      64          input_10[0][0]                   
                                                                 input_11[0][0]                   
__________________________________________________________________________________________________
graph_conv_6 (GraphConv)        (None, 784, 32)      1056        graph_conv_5[0][0]               
          

In [51]:
# Train model
validation_data = (X_val, y_val)
model.fit(X_train,
          y_train,
          batch_size=batch_size,
          validation_data=validation_data,
          epochs=epochs)




Train on 50000 samples, validate on 10000 samples
Epoch 1/1
50000/50000 [==============================] - 747s 15ms/step - loss: 1.0370 - acc: 0.7280 - val_loss: 0.4379 - val_acc: 0.8749


In [53]:

# Evaluate model
print('Evaluating model.')
eval_results = model.evaluate(X_test,
                              y_test,
                              batch_size=batch_size)
print('Done.\n'
      'Test loss: {}\n'
'Test acc: {}'.format(*eval_results))

Evaluating model.
10000/10000 [==============================] - 56s 6ms/step
Done.
Test loss: 0.41450210615992544
Test acc: 0.8805000030994415
